# DataExample

In [1]:
import pandas as pd

In [2]:
panpepdir = "/Users/kyoheikoyama/workspace/PanPep"

In [3]:
# !wc ./Data/majority_testing_dataset.csv

In [4]:
pd.read_csv(f"{panpepdir}/Data/Example_majority.csv")['Label'].value_counts()

1          11616
0          11616
Unknown     5230
Name: Label, dtype: int64

In [5]:
!head {panpepdir}/Data/Example_majority.csv

Peptide,CDR3,Label
ATDALMTGY,CAISESQGNTEAFF,1
ATDALMTGY,CAISEDRALVSYTF,1
ATDALMTGY,CAISEDRALNEQFF,1
ATDALMTGY,CAVQPGQGMQPQHF,1
ATDALMTGY,CAISEGAMGNQPQHF,1
ATDALMTGY,CAISDQTSGNQPQHF,1
ATDALMTGY,CAISDAGAGNQPQHF,1
ATDALMTGY,CAISEGTGGHQPQHF,1
ATDALMTGY,CAISESSAGNQPQHF,1


In [7]:
!wc {panpepdir}/Data/Example_majority.csv

   28463   28463  829060 /Users/kyoheikoyama/workspace/PanPep/Data/Example_majority.csv


In [7]:
!head -n 50000000 ./Data/Example_majority.csv > ./Data/Small_majority.csv

/bin/bash: ./Data/Small_majority.csv: No such file or directory


In [8]:
%%time
# !python PanPep.py --learning_setting majority \
#     --update_step_test 10 \
#     --input ./Data/Small_majority.csv \
#     --output ./Output/Example_majority_output.csv 


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


# Create data

In [9]:

## MCPAS
df = pd.read_parquet("/Users/kyoheikoyama/workspace/TCRPrediction/data/benchmark_tpp1_mcpas_test.parquet")
df_clean_mcpas =  df.rename(
    columns={'tcrb':'CDR3', 'peptide':'Peptide', 'sign':'Label'}
)[['Peptide','CDR3','Label',]].sort_values(by=['Peptide'])
# .drop(columns=['Label'])
# df_clean_mcpas.to_csv('./Data/benchmark_tpp1_mcpas.csv', index=None)

## VDJDB
df = pd.read_parquet("/Users/kyoheikoyama/workspace/TCRPrediction/data/benchmark_tpp1_vdjdb_test.parquet")
df_clean_vdjdb =  df.rename(
    columns={'tcrb':'CDR3', 'peptide':'Peptide', 'sign':'Label'}
)[['Peptide','CDR3','Label',]].sort_values(by=['Peptide'])
# .drop(columns=['Label'])
# df_clean_vdjdb.to_csv('./Data/benchmark_tpp1_vdjdb.csv', index=None)

# Zero-shot

In [10]:
%%time

!python PanPep.py --learning_setting zero-shot --update_step_test 100 \
    --input ./Data/benchmark_tpp1_vdjdb.csv --output ./Output/benchmark_tpp1_vdjdb_output.csv

!python PanPep.py --learning_setting zero-shot --update_step_test 100 \
    --input ./Data/benchmark_tpp1_mcpas.csv --output ./Output/benchmark_tpp1_mcpas_output.csv 



python: can't open file 'PanPep.py': [Errno 2] No such file or directory
python: can't open file 'PanPep.py': [Errno 2] No such file or directory
CPU times: user 7.27 ms, sys: 12.7 ms, total: 19.9 ms
Wall time: 333 ms


In [11]:
from sklearn.metrics import roc_auc_score

ypred_vdjdb = pd.read_csv(f"{panpepdir}/Output/benchmark_tpp1_vdjdb_output.csv")
ypred_mcpas = pd.read_csv(f"{panpepdir}/Output/benchmark_tpp1_mcpas_output.csv")

# Majority

In [12]:
def add_tcrpep(df_mcpas_test):
    df_mcpas_test['tcr_pep'] = df_mcpas_test['tcra'] + ":" + df_mcpas_test['tcrb'] + ":"  + df_mcpas_test['peptide']
#     df_mcpas_test['tcr_pep'] = df_mcpas_test['tcrb'] + ":"  + df_mcpas_test['peptide']
    return df_mcpas_test



In [13]:
import pandas as pd

## MCPAS


def create_data(dataname):
    df_test = pd.read_parquet(f"/Users/kyoheikoyama/workspace/TCRPrediction/data/benchmark_tpp1_{dataname}_test.parquet")
    df_test.sign = "Unknown"
    df_train = pd.read_parquet(f"/Users/kyoheikoyama/workspace/TCRPrediction/data/benchmark_tpp1_{dataname}_train.parquet")
    
    df = pd.concat([df_train, df_test])
    
    df_clean_mcpas =  df.rename(
        columns={'tcrb':'CDR3', 'peptide':'Peptide', 'sign':'Label'}
    )[['Peptide','CDR3','Label',]].sort_values(by=['Peptide', 'Label'])
    
    #df_clean_mcpas.to_csv(f'./Data/benchmark_tpp1_majority_{dataname}.csv', index=None)

dataname = 'mcpas'
create_data(dataname)

dataname = 'vdjdb'
create_data(dataname)


In [14]:
%%time
# !python PanPep.py --learning_setting majority --update_step_test 200 --input ./Data/benchmark_tpp1_majority_vdjdb.csv --output ./Output/benchmark_tpp1_vdjdb_output_majority.csv

# !python PanPep.py --learning_setting majority --update_step_test 200 --input ./Data/benchmark_tpp1_majority_mcpas.csv --output ./Output/benchmark_tpp1_mcpas_output_majority.csv 


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.15 µs


In [15]:
df_mcpas_train = pd.DataFrame(pd.read_pickle(f"../../external_data/ERGO-II/Samples/mcpas_train_samples.pickle")).query('tcra!="UNK"') #.query('sign==1')
df_mcpas_train = add_tcrpep(df_mcpas_train)
df_vdjdb_train = pd.DataFrame(pd.read_pickle(f"../../external_data/ERGO-II/Samples/vdjdb_no10x_train_samples.pickle")).query('tcra!="UNK"') #.query('sign==1')
df_vdjdb_train = add_tcrpep(df_vdjdb_train)



# MCPAS

In [16]:
dataname = 'mcpas'

ypred = pd.read_csv(f"{panpepdir}/Output/benchmark_tpp1_{dataname}_output_majority.csv")
y_test = pd.read_parquet(f"/Users/kyoheikoyama/workspace/TCRPrediction/data/benchmark_tpp1_{dataname}_test.parquet")

ypred = ypred.drop_duplicates(['CDR3', 'Peptide'])
y_test = pd.merge(y_test, ypred, 
         left_on=['tcrb', 'peptide'], 
         right_on=['CDR3', 'Peptide'], 
         how='left')
y_test.Score = [float(s.replace('tensor(','').replace(')','')) if isinstance(s, str) else s.item() for s in y_test.Score]

y_test = add_tcrpep(y_test)

y_test = y_test[~y_test.tcr_pep.isin(df_mcpas_train.tcr_pep)]

print(y_test.shape, ypred.shape)


(4729, 15) (4321, 3)


In [17]:
from sklearn.metrics import roc_auc_score, average_precision_score

print("roc_auc_score", roc_auc_score(y_test.sign, y_test.Score))

print("average_precision_score",average_precision_score(y_test.sign, y_test.Score))

roc_auc_score 0.8373969080247107
average_precision_score 0.4518658630841503


In [18]:
y_test.shape

(4729, 15)

# VDJDB

In [19]:
dataname = 'vdjdb'

ypred = pd.read_csv(f"{panpepdir}/Output/benchmark_tpp1_{dataname}_output_majority.csv")
y_test = pd.read_parquet(f"/Users/kyoheikoyama/workspace/TCRPrediction/data/benchmark_tpp1_{dataname}_test.parquet")
print(y_test.shape, ypred.shape)

ypred = ypred.drop_duplicates(['CDR3', 'Peptide'])
y_test = pd.merge(y_test, ypred, 
         left_on=['tcrb', 'peptide'], 
         right_on=['CDR3', 'Peptide'], 
         how='left')

print(y_test.shape, ypred.shape)


y_test.Score = [float(s.replace('tensor(','').replace(')','')) if isinstance(s, str) else s.item() for s in y_test.Score]

y_test = add_tcrpep(y_test)


y_test = y_test[~y_test.tcr_pep.isin(df_vdjdb_train.tcr_pep)]

print(y_test.shape, ypred.shape)

print("roc_auc_score",roc_auc_score(y_test.sign, y_test.Score))

print("average_precision_score",average_precision_score(y_test.sign, y_test.Score))




(4485, 11) (4485, 3)
(4485, 14) (3360, 3)
(4010, 15) (3360, 3)
roc_auc_score 0.9009326715410326
average_precision_score 0.6435366106466862


In [20]:
y_test.shape, ypred.shape

((4010, 15), (3360, 3))

In [21]:
pd.value_counts(y_test.tcr_pep.isin(df_vdjdb_train.tcr_pep))

False    4010
Name: tcr_pep, dtype: int64